In [1]:
import torch
from utils.loader import DicomDataset3D
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import utils.notebooks as nb

train_dataloader = DataLoader(DicomDataset3D("data/train.csv"), batch_size=2)
test_dataloader = DataLoader(DicomDataset3D("data/test.csv"), batch_size=2)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
train_features, train_labels = next(iter(train_dataloader))
print(f'inputs shape: {train_features.shape}')
print(f'labels shape: {train_labels.shape}')
feat1, label1 = train_features[0], train_labels[0]
print(f'inputs type: {feat1.dtype}')
print(f'labels type: {label1.dtype}')

inputs shape: torch.Size([2, 1, 88, 256, 256])
labels shape: torch.Size([2, 1, 88, 256, 256])
inputs type: torch.float32
labels type: torch.float32


### Single ConvBlock

In [3]:
from models.WNet import ConvBlock
input = train_features.to(device)

conv_block = ConvBlock(1, 8)
# with torch.autocast(device_type=device, dtype=torch.bfloat16):
x = conv_block(input)
x.shape

torch.Size([2, 8, 88, 256, 256])

### Encoder

In [4]:
from models.WNet import Encoder
from utils.loader import DicomDataset3D
from torch.utils.data import DataLoader
import torch
train_dataloader = DataLoader(DicomDataset3D("data/train.csv"), batch_size=2)
train_features, train_labels = next(iter(train_dataloader))
device = "cuda" if torch.cuda.is_available() else "cpu"
input = train_features.to(device)
layers = [1, 8, 16, 32]

encoder = Encoder(layers)
x = encoder(input)
for ftr in x: print(ftr.shape)

torch.Size([2, 8, 88, 256, 256])
torch.Size([2, 16, 44, 128, 128])
torch.Size([2, 32, 22, 64, 64])


### Decoder

In [5]:
from models.WNet import ConvBlock, Encoder, Decoder
import torch
from utils.loader import DicomDataset3D
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
layers = [1, 4, 8, 16]
train_dataloader = DataLoader(DicomDataset3D("data/train.csv"), batch_size=2)
train_features, train_labels = next(iter(train_dataloader))
input = train_features.to(device)

encoder = Encoder(layers)
decoder = Decoder(list(reversed(layers[1:])))

ftrs = encoder(input)
x = decoder(ftrs[::-1][0], ftrs[::-1][1:])
x.shape

torch.Size([2, 4, 88, 256, 256])

## U-Net

In [6]:
from models.WNet import UNet
import torch
from utils.loader import DicomDataset3D
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
layers = [1, 8, 16]
train_dataloader = DataLoader(DicomDataset3D("data/train.csv"), batch_size=1)
train_features, train_labels = next(iter(train_dataloader))
input = train_features.to(device)

unet = UNet(layers=layers)
x = unet(input)
x.shape

torch.Size([1, 1, 88, 256, 256])

## W-Net

In [2]:
from models.WNet import WNet
import torch
from utils.loader import DicomDataset3D
from torch.utils.data import DataLoader
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"
layers = [1, 8, 16, 32]
train_dataloader = DataLoader(DicomDataset3D("data/train.csv"), batch_size=2)
train_features, train_labels = next(iter(train_dataloader))
input = train_features.to(device)

wnet = WNet(layers=layers)
x = wnet(input)
x.shape

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.HalfTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor